<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#!/usr/bin/env python3
# ═══════════════════════════════════════════════════════════════════════════
# MY-AI-GIZMO - NO UI MODIFICATIONS (CLEAN INSTALL)
# ═══════════════════════════════════════════════════════════════════════════

import os
import sys
import subprocess
import shutil
import time
import re
from pathlib import Path

print("🚀 MY-AI-GIZMO - CLEAN INSTALLER (NO UI MODS)")
print("="*70)

# ═══════════════════════════════════════════════════════════════════════════
# HELPERS
# ═══════════════════════════════════════════════════════════════════════════

def run_cmd(cmd, check=False, quiet=True, timeout=None):
    try:
        if quiet:
            return subprocess.run(cmd, check=check, capture_output=True, text=True, timeout=timeout)
        else:
            return subprocess.run(cmd, check=check, timeout=timeout)
    except:
        return None

def extract_urls(text):
    urls = {}
    public_match = re.search(r'https://[a-z0-9]+\.gradio\.live', text, re.IGNORECASE)
    if public_match:
        urls['public'] = public_match.group(0)
    local_match = re.search(r'http://127\.0\.0\.1:\d+', text)
    if local_match:
        urls['local'] = local_match.group(0)
    return urls

def print_url_box(local_url, public_url, hw):
    box = "="*70
    print("\n" + box)
    print("🎉"*35)
    print(box)
    print("✅ SERVER IS RUNNING!")
    print(box)

    if public_url:
        print(f"\n🌐 PUBLIC URL (CLICK THIS):\n")
        print(f"   {public_url}\n")

    if local_url:
        print(f"🏠 LOCAL URL:\n")
        print(f"   {local_url}\n")

    print(box)
    print(f"💾 Chats saved to Google Drive")
    print(f"🖥️  Hardware: {hw}")
    print(f"⏹️  Press Ctrl+C to stop")
    print(box + "\n")

# ═══════════════════════════════════════════════════════════════════════════
# PATHS
# ═══════════════════════════════════════════════════════════════════════════

DRIVE_BASE = Path("/content/drive/MyDrive/MY-AI-Gizmo")
REPO_DIR = DRIVE_BASE / "MY-AI-Gizmo-working"
MODELS_DIR = DRIVE_BASE / "models"
USER_DATA_DIR = DRIVE_BASE / "user_data"
MODEL_FILE = MODELS_DIR / "llama-2-7b.Q4_K_M.gguf"

# ═══════════════════════════════════════════════════════════════════════════
# STEP 1: MOUNT DRIVE
# ═══════════════════════════════════════════════════════════════════════════

print("\n📂 Step 1: Mounting Google Drive...")

try:
    from google.colab import drive
    mydrive = Path("/content/drive/MyDrive")

    if mydrive.exists() and mydrive.is_dir():
        print("   ✅ Already mounted")
    else:
        drive.mount('/content/drive', force_remount=True)
        print("   ✅ Mounted")

    if not mydrive.exists():
        raise Exception("Mount failed")

except Exception as e:
    print(f"   ❌ Error: {e}")
    raise SystemExit(1)

for d in [DRIVE_BASE, MODELS_DIR, USER_DATA_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ═══════════════════════════════════════════════════════════════════════════
# STEP 2-3: SYSTEM + REPO
# ═══════════════════════════════════════════════════════════════════════════

print("\n📦 Step 2: System packages...")
run_cmd(["apt-get", "update", "-qq"])
run_cmd(["apt-get", "install", "-y", "-qq", "build-essential", "cmake", "git", "wget"])
print("   ✅ Done")

print("\n📥 Step 3: Repository...")

# Clean install if UI was previously broken
if (REPO_DIR / "modules" / "ui.py").exists():
    with open(REPO_DIR / "modules" / "ui.py", 'r', encoding='utf-8') as f:
        if '# COLAB_FIX' in f.read():
            print("   🔄 Removing old broken UI, fresh clone needed...")
            if REPO_DIR.exists():
                shutil.rmtree(REPO_DIR)

if (REPO_DIR / "server.py").exists():
    print("   ✅ Already exists (clean)")
else:
    REPO_DIR.parent.mkdir(parents=True, exist_ok=True)
    os.chdir(REPO_DIR.parent)
    run_cmd(["git", "clone", "https://github.com/gitleon8301/MY-AI-Gizmo-working.git"])
    print("   ✅ Cloned fresh")

os.chdir(REPO_DIR)

# ═══════════════════════════════════════════════════════════════════════════
# STEP 4: PACKAGES - FAST WITH PRE-BUILT WHEELS
# ═══════════════════════════════════════════════════════════════════════════

print("\n🐍 Step 4: Installing packages...")
print("   (Using pre-built wheels - fast!)")

# Core
print("   📦 Core packages...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q",
               "setuptools", "wheel", "numpy", "requests", "tqdm", "pyyaml"],
              check=False, timeout=120)

# llama-cpp-python PRE-BUILT (CPU)
print("   📦 llama-cpp-python (pre-built, ~30s)...")
subprocess.run([sys.executable, "-m", "pip", "install",
               "llama-cpp-python[server]",
               "--extra-index-url", "https://abetlen.github.io/llama-cpp-python/whl/cpu"],
              check=False, timeout=180)

# Main packages
print("   📦 AI packages...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q",
               "torch", "transformers", "gradio>=3.50.0", "accelerate",
               "markdown", "Pillow", "safetensors", "sentencepiece", "protobuf"],
              check=False, timeout=300)

# Extensions
print("   📦 Extension packages...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q",
               "flask_cloudflared"],
              check=False, timeout=60)

print("   ✅ All packages installed!")

# ═══════════════════════════════════════════════════════════════════════════
# STEP 5: NO UI MODIFICATIONS - LEAVE IT CLEAN!
# ═══════════════════════════════════════════════════════════════════════════

print("\n✅ Step 5: Skipping UI modifications (using GitHub version as-is)")

# ═══════════════════════════════════════════════════════════════════════════
# STEP 6: DATA SETUP
# ═══════════════════════════════════════════════════════════════════════════

print("\n🔗 Step 6: Linking user data...")
local_data = REPO_DIR / "user_data"
if local_data.exists() and not local_data.is_symlink():
    for item in local_data.rglob("*"):
        if item.is_file():
            rel = item.relative_to(local_data)
            dest = USER_DATA_DIR / rel
            dest.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(item, dest)
    shutil.rmtree(local_data)
if not local_data.exists():
    local_data.symlink_to(USER_DATA_DIR)
for sub in ["logs","logs/chat","logs/instruct","presets","characters"]:
    (USER_DATA_DIR / sub).mkdir(parents=True, exist_ok=True)
print("   ✅ Linked")

print("\n⬇️  Step 7: Model check...")
if MODEL_FILE.exists():
    print(f"   ✅ Found ({MODEL_FILE.stat().st_size/(1024**3):.2f} GB)")
else:
    print("   📥 Downloading model...")
    MODEL_FILE.parent.mkdir(parents=True, exist_ok=True)
    subprocess.run(["wget","-q","--show-progress","-O",str(MODEL_FILE),
                   "https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q4_K_M.gguf"],
                  timeout=900)
    print("   ✅ Downloaded")

print("\n🔗 Step 8: Linking models...")
repo_models = REPO_DIR / "models"
if not repo_models.is_symlink():
    if repo_models.exists():
        shutil.rmtree(repo_models)
    repo_models.symlink_to(MODELS_DIR)
print("   ✅ Linked")

print("\n🖥️  Step 9: Hardware check...")
has_gpu = False
try:
    import torch
    has_gpu = torch.cuda.is_available()
    if has_gpu:
        print(f"   ✅ GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("   ℹ️  CPU mode")
except:
    print("   ℹ️  CPU mode")

hw = "GPU" if has_gpu else "CPU"

# ═══════════════════════════════════════════════════════════════════════════
# STEP 10: LAUNCH - ADD --old-colors FLAG TO USE LEGACY GRADIO THEME
# ═══════════════════════════════════════════════════════════════════════════

print("\n🚀 Step 10: Launching server...")
print(f"   Hardware: {hw}")
print(f"   Model: {MODEL_FILE.name}")

cmd = [sys.executable, "server.py"]
if has_gpu:
    cmd.extend(["--gpu-layers", "35"])
cmd.extend(["--cpu", "--threads","4","--listen","--listen-host","0.0.0.0",
            "--share","--model",str(MODEL_FILE),"--loader","llama.cpp",
            "--old-colors"])  # Use old Gradio theme - avoids the theme code issues!

print("\n" + "="*70)
print("🌐 STARTING GRADIO SERVER")
print("="*70)
print("Using --old-colors flag to avoid theme issues")
print("Watching for URLs...")
print("="*70 + "\n")

os.chdir(REPO_DIR)

found_urls = {'local': None, 'public': None}
url_box_printed = False

try:
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                              universal_newlines=True, bufsize=1)

    for line in process.stdout:
        print(line, end='')

        line_urls = extract_urls(line)

        if 'local' in line_urls and not found_urls['local']:
            found_urls['local'] = line_urls['local']
            print(f"\n✅ LOCAL URL: {found_urls['local']}")

        if 'public' in line_urls and not found_urls['public']:
            found_urls['public'] = line_urls['public']
            print(f"\n🌐 PUBLIC URL: {found_urls['public']}")

        if found_urls['public'] and not url_box_printed:
            url_box_printed = True
            print_url_box(found_urls['local'], found_urls['public'], hw)

    process.wait()

except KeyboardInterrupt:
    print("\n\n" + "="*70)
    print("⏹️  Server stopped")
    if found_urls['public']:
        print(f"📝 Public URL: {found_urls['public']}")
    if found_urls['local']:
        print(f"📝 Local URL: {found_urls['local']}")
    print(f"💾 Data: {USER_DATA_DIR}")
    print("="*70)

except Exception as e:
    print(f"\n❌ ERROR: {e}")
    import traceback
    traceback.print_exc()

print("\n✅ Complete!")